In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from sklearn.externals import joblib

In [39]:
incident_df = pd.read_excel('incident_dataset.xlsx', sheet_name='Sheet2')

incident_df['Exception']  = incident_df['Exception'].fillna('NA')
incident_df['CloudHealthIndex']  = incident_df['CloudHealthIndex'].fillna('NA')


incidentTypesDict = dict(zip(incident_df['Label'].unique(), np.arange(incident_df['Label'].unique().size)))
exceptionTypes = dict(zip(incident_df['Exception'].unique(), np.arange(incident_df['Exception'].unique().size)))
cloudHealthValues = dict(zip(incident_df['CloudHealthIndex'].unique(), np.arange(incident_df['CloudHealthIndex'].unique().size)))
features_vars = list(incident_df.columns.drop(['Label']).values)


print('incidentTypesDict  = ', incidentTypesDict, '\n')
print('exceptionTypes  = ', exceptionTypes, '\n')
print('cloudHealthValues  = ', cloudHealthValues, '\n')
print('features_vars  = ', features_vars, '\n')

incident_df.head()


incidentTypesDict  =  {'NetworkIssue': 0, 'NoIssue': 1, 'DatabaseConnection': 2, 'CommunityHealthIssue': 3, 'Community-FromtDoorNotaccessible': 4, 'InvoiceIssue': 5, 'UnsearchableWorkspaces': 6, 'DataLoad Failure': 7} 

exceptionTypes  =  {'spanning tree event': 0, 'NA': 1, 'JDBC-connection-permit-failure': 2, '[AWGenericException: java.lang.IllegalStateException, Export-webservice-ConnectionTimeout]': 3, '[ScheduledTsak-ArchesBatchPublishInThisRealm-Failure, Arches Schema version mismatch]': 4, '[OutOfMemoryException, GT Nodes restarting]': 5} 

cloudHealthValues  =  {'FAIR': 0, 'GOOD': 1, 'POOR': 2, 'NA': 3, 'CRITICAL': 4} 

features_vars  =  ['Date', 'AvgBackgroundQ', 'AvgThreadPoolSize', 'AvgWorkflowQ', 'CatalogSearchTime', 'IncreasingBGQueueTrend', 'IncreasingThreadTrend', 'IncreasingWFQueueTrend', 'Exception', 'LogSizeVolumePercent', 'NetworkConnectivitySNV-US1', 'IsProductReleased', 'UiNodeThreadsCount', 'CloudHealthIndex'] 



,Date,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,IncreasingBGQueueTrend,IncreasingThreadTrend,IncreasingWFQueueTrend,Exception,LogSizeVolumePercent,NetworkConnectivitySNV-US1,IsProductReleased,UiNodeThreadsCount,CloudHealthIndex,Label
0,2017-01-01 00:00:00,296.0,9.0,14.0,16.0,0,0,0,spanning tree event,39.0,1.0,1.0,13.0,FAIR,NetworkIssue
1,2017-01-01 00:30:00,284.0,9.0,8.0,15.0,0,0,0,spanning tree event,40.0,1.0,1.0,13.0,GOOD,NetworkIssue
2,2017-01-01 01:00:00,NaN,9.0,14.0,13.0,0,0,0,spanning tree event,26.0,1.0,0.0,19.0,POOR,NetworkIssue
3,2017-01-01 01:30:00,289.0,8.0,14.0,11.0,0,0,0,spanning tree event,39.0,1.0,0.0,12.0,NA,NetworkIssue
4,2017-01-01 02:00:00,304.0,10.0,10.0,10.0,0,0,0,spanning tree event,29.0,1.0,1.0,11.0,GOOD,NetworkIssue


### Pipeline and Estimator to PreProcess incident Dataset

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin


class Preprocessor(BaseEstimator, TransformerMixin):
    """Custom Preprocessing Estimator for the custom case
    
    """
    
    def __init___(self):
        pass
    
    def transform(self, df):

        exceptionTypes  =  {'spanning tree event': 0, 'NA': 1, 'JDBC-connection-permit-failure': 2, '[AWGenericException: java.lang.IllegalStateException, Export-webservice-ConnectionTimeout]': 3, '[ScheduledTsak-ArchesBatchPublishInThisRealm-Failure, Arches Schema version mismatch]': 4, '[OutOfMemoryException, GT Nodes restarting]': 5} 

        cloudHealthValues  =  {'FAIR': 0, 'GOOD': 1, 'POOR': 2, 'NA': 3, 'CRITICAL': 4} 

        features_vars  =  ['Date', 'AvgBackgroundQ', 'AvgThreadPoolSize', 'AvgWorkflowQ', 'CatalogSearchTime', 'IncreasingBGQueueTrend', 
                           'IncreasingThreadTrend', 'IncreasingWFQueueTrend', 'Exception', 'LogSizeVolumePercent', 'NetworkConnectivitySNV-US1', 
                           'IsProductReleased', 'UiNodeThreadsCount', 'CloudHealthIndex'] 
        
        df = df[features_vars]
        
        # Make all negative values to NaN so that it can be replaced with single value
        df['AvgBackgroundQ'][df['AvgBackgroundQ'] < 1] = np.NAN
        df['AvgThreadPoolSize'][df['AvgThreadPoolSize'] < 1] = np.NAN
        df['AvgWorkflowQ'][df['AvgWorkflowQ'] < 1] = np.NAN
        df['CatalogSearchTime'][df['CatalogSearchTime'] < 1] = np.NAN
        df['UiNodeThreadsCount'][df['UiNodeThreadsCount'] < 1] = np.NAN
        
        df = df.fillna(-999)
        
        df1 = df.apply(lambda rec : pd.Series({'Day' : rec['Date'].day, 
                      'Month' : rec['Date'].month, 
                      'Year' :  rec['Date'].year,
                      'Hour' : rec['Date'].hour, 
                      'Minute' : rec['Date'].minute, 
                      'Second' :  rec['Date'].second,
                      'DayOfYear' : rec['Date'].dayofyear, 
                      'DayOfWeek' : rec['Date'].dayofweek, 
                      'WeekOfYear' :  rec['Date'].weekofyear,
                      'WeekOfYear' : rec['Date'].weekofyear, 
                      'Quarter' : rec['Date'].quarter,
                      'IsWeekend' : int(rec['Date'].dayofweek > 4), 
                      'IsMonthStart' : int(rec['Date'].is_month_start),
                      'IsMonthEnd' :  int(rec['Date'].is_month_end)
                     }), axis = 1)
        
        df = pd.concat([df, df1], axis=1)

        df.replace({'CloudHealthIndex': cloudHealthValues}, inplace=True)
        df.replace({'Exception': exceptionTypes}, inplace=True)
        
        df = df[df.columns.drop(['Date'])]
        return df  #.as_matrix()
    
    def fit(self, df, y=None, **fit_params):
        
        return self;
        
        


In [4]:
# Get the y values from DataFram
y = incident_df['Label'].replace(incidentTypesDict)
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

In [5]:
preprocess = Preprocessor()

X = preprocess.transform(incident_df)
X.head()

,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,IncreasingBGQueueTrend,IncreasingThreadTrend,IncreasingWFQueueTrend,Exception,LogSizeVolumePercent,NetworkConnectivitySNV-US1,...,Hour,Minute,Second,DayOfYear,DayOfWeek,WeekOfYear,Quarter,IsWeekend,IsMonthStart,IsMonthEnd
0,296.0,9.0,14.0,16.0,0,0,0,0,39.0,1.0,...,0,0,0,1,6,52,1,1,1,0
1,284.0,9.0,8.0,15.0,0,0,0,0,40.0,1.0,...,0,30,0,1,6,52,1,1,1,0
2,-999.0,9.0,14.0,13.0,0,0,0,0,26.0,1.0,...,1,0,0,1,6,52,1,1,1,0
3,289.0,8.0,14.0,11.0,0,0,0,0,39.0,1.0,...,1,30,0,1,6,52,1,1,1,0
4,304.0,10.0,10.0,10.0,0,0,0,0,29.0,1.0,...,2,0,0,1,6,52,1,1,1,0


### Split the dataset in Training and Test data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.head()

,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,IncreasingBGQueueTrend,IncreasingThreadTrend,IncreasingWFQueueTrend,Exception,LogSizeVolumePercent,NetworkConnectivitySNV-US1,...,Hour,Minute,Second,DayOfYear,DayOfWeek,WeekOfYear,Quarter,IsWeekend,IsMonthStart,IsMonthEnd
551,282.0,11.0,13.0,14.0,0,0,0,1,28.0,0.0,...,11,30,0,12,3,2,1,0,0,0
595,294.0,9.0,12.0,12.0,0,0,0,1,35.0,0.0,...,9,30,0,13,4,2,1,0,0,0
617,293.0,9.0,8.0,13.0,0,0,0,1,28.0,0.0,...,20,30,0,13,4,2,1,0,0,0
376,289.0,8.0,11.0,12.0,0,0,0,1,27.0,0.0,...,20,0,0,8,6,1,1,1,0,0
262,300.0,11.0,13.0,11.0,0,0,0,1,38.0,0.0,...,11,0,0,6,4,1,1,0,0,0


### Train the algorithm using Classifier

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.932806324110672

### Save the Model in file

In [8]:
import pickle
import dill as pickle

filename = 'model_v3.pk'

with open('models/'+filename, 'wb') as file:
    pickle.dump(knn, file)

In [9]:
knn.predict(X_test)

array([1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 6, 1, 1, 1, 0, 1, 5,
       1, 1, 1, 1, 1, 1, 3, 5, 1, 1, 1, 1, 1, 1, 1, 4, 6, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 5, 1, 1, 1, 4,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 7, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 3, 6, 1, 1, 0, 1,
       1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 6, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 6, 1, 1, 4, 1, 6, 6, 7, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 5, 1, 1, 6, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 4, 1, 5, 0, 1, 1, 1, 1], dtype=int64)

In [10]:
X_train.shape

(756, 26)

In [11]:
req = incident_df[4:6].to_json(orient='records')

In [12]:
reqDf = incident_df.iloc[15:17]
reqDf = reqDf[reqDf.columns.drop(['Label'])]
reqDf
j1 = reqDf.to_json(orient='records')
j1

'[{"Date":1483255800000,"AvgBackgroundQ":null,"AvgThreadPoolSize":11.0,"AvgWorkflowQ":8.0,"CatalogSearchTime":16.0,"IncreasingBGQueueTrend":0,"IncreasingThreadTrend":0,"IncreasingWFQueueTrend":0,"Exception":"spanning tree event","LogSizeVolumePercent":32.0,"NetworkConnectivitySNV-US1":1.0,"IsProductReleased":0.0,"UiNodeThreadsCount":18.0,"CloudHealthIndex":"FAIR"},{"Date":1483257600000,"AvgBackgroundQ":311.0,"AvgThreadPoolSize":9.0,"AvgWorkflowQ":14.0,"CatalogSearchTime":14.0,"IncreasingBGQueueTrend":0,"IncreasingThreadTrend":0,"IncreasingWFQueueTrend":0,"Exception":"spanning tree event","LogSizeVolumePercent":32.0,"NetworkConnectivitySNV-US1":1.0,"IsProductReleased":1.0,"UiNodeThreadsCount":16.0,"CloudHealthIndex":"GOOD"}]'

In [13]:
j1_dff = pd.read_json(j1)
j1_dff

,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,CloudHealthIndex,Date,Exception,IncreasingBGQueueTrend,IncreasingThreadTrend,IncreasingWFQueueTrend,IsProductReleased,LogSizeVolumePercent,NetworkConnectivitySNV-US1,UiNodeThreadsCount
0,NaN,11,8,16,FAIR,2017-01-01 07:30:00,spanning tree event,0,0,0,0,32,1,18
1,311.0,9,14,14,GOOD,2017-01-01 08:00:00,spanning tree event,0,0,0,1,32,1,16


In [14]:
preprocess = Preprocessor()
m1 = preprocess.transform(j1_dff)
knn.predict(m1)

array([1, 0], dtype=int64)

### Training with Random Forest Classifiers

In [15]:
param_grid = {"randomforestclassifier__n_estimators" : [10, 20, 30],
             "randomforestclassifier__max_depth" : [None, 6, 8, 10],
             "randomforestclassifier__max_leaf_nodes": [None, 5, 10, 20], 
             "randomforestclassifier__min_impurity_split": [0.1, 0.2, 0.3]}


In [43]:
class IncidentPreprocessor(BaseEstimator, TransformerMixin):
    """Custom Preprocessing Estimator for the custom case
    
    """
    
    def __init___(self):
        pass
    
    def transform(self, df):

        exceptionTypes  =  {'spanning tree event': 0, 'NA': 1, 'JDBC-connection-permit-failure': 2, '[AWGenericException: java.lang.IllegalStateException, Export-webservice-ConnectionTimeout]': 3, '[ScheduledTsak-ArchesBatchPublishInThisRealm-Failure, Arches Schema version mismatch]': 4, '[OutOfMemoryException, GT Nodes restarting]': 5} 

        cloudHealthValues  =  {'FAIR': 0, 'GOOD': 1, 'POOR': 2, 'NA': 3, 'CRITICAL': 4} 

        features_vars  =  ['Date','AvgBackgroundQ', 'AvgThreadPoolSize', 'AvgWorkflowQ', 'CatalogSearchTime', 'IncreasingBGQueueTrend', 
                           'IncreasingThreadTrend', 'IncreasingWFQueueTrend', 'Exception', 'LogSizeVolumePercent', 'NetworkConnectivitySNV-US1', 
                           'IsProductReleased', 'UiNodeThreadsCount', 'CloudHealthIndex'] 
        
        df = df[features_vars]
        
        # Make all negative values to NaN so that it can be replaced with single value
        df['AvgBackgroundQ'][df['AvgBackgroundQ'] < 1] = np.NAN
        df['AvgThreadPoolSize'][df['AvgThreadPoolSize'] < 1] = np.NAN
        df['AvgWorkflowQ'][df['AvgWorkflowQ'] < 1] = np.NAN
        df['CatalogSearchTime'][df['CatalogSearchTime'] < 1] = np.NAN
        df['UiNodeThreadsCount'][df['UiNodeThreadsCount'] < 1] = np.NAN
        
        df = df.fillna(-999)
        
        df1 = df.apply(lambda rec : pd.Series({'Day' : rec['Date'].day, 
                      'Month' : rec['Date'].month, 
                      'Year' :  rec['Date'].year,
                      'Hour' : rec['Date'].hour, 
                      'Minute' : rec['Date'].minute, 
                      'Second' :  rec['Date'].second,
                      'DayOfYear' : rec['Date'].dayofyear, 
                      'DayOfWeek' : rec['Date'].dayofweek, 
                      'WeekOfYear' :  rec['Date'].weekofyear,
                      'WeekOfYear' : rec['Date'].weekofyear, 
                      'Quarter' : rec['Date'].quarter,
                      'IsWeekend' : int(rec['Date'].dayofweek > 4), 
                      'IsMonthStart' : int(rec['Date'].is_month_start),
                      'IsMonthEnd' :  int(rec['Date'].is_month_end)
                     }), axis = 1)
        
        df = pd.concat([df, df1], axis=1)

        df.replace({'CloudHealthIndex': cloudHealthValues}, inplace=True)
        df.replace({'Exception': exceptionTypes}, inplace=True)
        
        df = df[df.columns.drop(['Date'])]
        return df  #.as_matrix()
    
    def fit(self, df, y=None, **fit_params):
        
        return self;


In [44]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(IncidentPreprocessor(), RandomForestClassifier())
pipe

Pipeline(memory=None,
     steps=[('incidentpreprocessor', IncidentPreprocessor()), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [45]:
from sklearn.model_selection import train_test_split, GridSearchCV

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)
grid

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('incidentpreprocessor', IncidentPreprocessor()), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__n_estimators': [10, 20, 30], 'randomforestclassifier__max_depth': [None, 6, 8, 10], 'randomforestclassifier__max_leaf_nodes': [None, 5, 10, 20], 'randomforestclassifier__min_impurity_split': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [52]:
X = incident_df
y = incident_df['Label'].replace(incidentTypesDict)

X_train, X_test, y_train, y_test = train_test_split(X,y)

grid.fit(X_train,y_train)
# X.head()
# y.head()

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

...

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\softwares\python\python36-32\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarni

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('incidentpreprocessor', IncidentPreprocessor()), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
      ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__n_estimators': [10, 20, 30], 'randomforestclassifier__max_depth': [None, 6, 8, 10], 'randomforestclassifier__max_leaf_nodes': [None, 5, 10, 20], 'randomforestclassifier__min_impurity_split': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [53]:
print("Best parameters: {}".format(grid.best_params_))

Best parameters: {'randomforestclassifier__max_depth': 8, 'randomforestclassifier__max_leaf_nodes': 20, 'randomforestclassifier__min_impurity_split': 0.1, 'randomforestclassifier__n_estimators': 20}


In [71]:
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Test set score: 0.98


In [75]:
import pickle
import dill as pickle

filename1 = 'model_random_forest_v1.pk'

with open('models/' + filename1, 'wb') as file:
    pickle.dump(grid, file)
    

In [76]:
## Save model using joblib SKLearn
fname = 'model_random_forest_v2.pk'
joblib.dump(grid, 'models/' + fname)

['models/model_random_forest_v2.pk']

In [74]:
# grid.predict(X_test)